In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# 대상 기업 정보 불러오기
fs_all = pd.read_excel('data/fs-account.xlsx', dtype = str)

In [72]:
# dart_code_list 불러오기
dart_code_list = pd.read_json('data/dart_code_list.json', dtype = str)
dart_code_list

,고유번호,정식명칭,종목코드
0,00260985,한빛네트,036720
1,00264529,엔플렉스,040130
2,00358545,동서정보기술,055000
3,00231567,애드모바일,032600
4,00359614,리더컴,056140
...,...,...,...
3787,00317104,라이온켐텍,171120
3788,00651901,에어부산,298690
3789,00661847,화인베스틸,133820
3790,01428203,케이씨씨글라스,344820


In [ ]:
# 1. 자산
# 1) 자산 총계

fs_data = fs_all.copy()

bs = fs_data['sj_div'] == 'BS'

asset = bs & (
    (fs_data['account_id'] == 'ifrs-full_Assets') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & fs_data['account_nm'].str.contains('자산총계')
    )
if len(fs_data[asset]) > 1 :
    asset = bs & fs_data['account_id'].str.contains('ifrs-full_Assets')

df1 = fs_data.loc[asset, 'corp_code'].to_list()

# 2) 유동자산
current_asset = bs & (fs_data['account_id'] == 'ifrs-full_CurrentAssets')
df2 = fs_data.loc[current_asset, 'corp_code'].to_list()

# 3) 비유동자산
non_current_asset = bs & (fs_data['account_id'] == 'ifrs-full_NoncurrentAssets')
df3 = fs_data.loc[non_current_asset, 'corp_code'].to_list()

# 4) 현금 및 현금성자산
cash = bs & (fs_data['account_id'] == 'ifrs-full_CashAndCashEquivalents')
df4 = fs_data.loc[cash, 'corp_code'].to_list()

# 5) 매출채권
rcvb_idx = bs & (
    fs_data['account_id'].str.contains('ifrs-full_TradeAndOtherCurrentReceivables') |
    fs_data['account_id'].str.contains('ifrs-full_TradeReceivables') |
    fs_data['account_id'].str.contains('ifrs-full_CurrentTradeReceivables') |
    fs_data['account_id'].str.contains('dart_ShortTermTradeReceivable') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & fs_data['account_nm'].str.contains('매출채권')
)

# else :
receivable = rcvb_idx
df5 = fs_data.loc[receivable, 'corp_code'].to_list()

# 6) 재고자산
inventory = bs & (
    fs_data['account_id'].str.contains('ifrs-full_Inventories') |
    fs_data['account_id'].str.contains('-표준계정코드 미사용-') & (fs_data['account_nm'] == '재고자산')
)

if len(fs_data[inventory]) > 1 :
    inventory = bs & fs_data['account_id'].str.contains('ifrs-full_Inventories')

df6 = fs_data.loc[inventory, 'corp_code'].to_list()

# 2. 자본
# 1) 자기자본
equity = bs & (fs_data['account_id'] == 'ifrs-full_Equity')
df7 = fs_data.loc[equity, 'corp_code'].to_list()
# 2) 이익잉여금
retained_earning = bs & (fs_data['account_id'] == 'ifrs-full_RetainedEarnings')
df8 = fs_data.loc[retained_earning, 'corp_code'].to_list()

# 3. 부채
# 1) 부채총계
liability = bs & (fs_data['account_id'] == 'ifrs-full_Liabilities')
df9 = fs_data.loc[liability, 'corp_code'].to_list()
# 2) 유동부채
current_liability = bs & (fs_data['account_id'] == 'ifrs-full_CurrentLiabilities')
df10 = fs_data.loc[current_liability, 'corp_code'].to_list()
# 3) 비유동부채
non_current_liability = bs & (fs_data['account_id'] == 'ifrs-full_NoncurrentLiabilities')
df11 = fs_data.loc[non_current_liability, 'corp_code'].to_list()

# 매출액
revenue = (fs_data['account_id'] == 'ifrs-full_Revenue')
df12 = fs_data.loc[revenue, 'corp_code'].to_list()

# 영업이익
operating_income = (fs_data['sj_div'] == 'CIS') &(
    fs_data['account_id'].str.contains('ifrs-full_ProfitLossFromOperatingActivities') |
    fs_data['account_id'].str.contains('dart_OperatingIncomeLoss')
)

df13 = fs_data.loc[operating_income, 'corp_code'].to_list()

# 당기순이익
net_income = (
    fs_data['sj_div'] == 'CIS') & (
     fs_data['account_id'] == 'ifrs-full_ProfitLoss'   
    )
df14 = fs_data.loc[net_income, 'corp_code'].to_list()


In [77]:
print('df1', len(df1))
print('df2', len(df2))
print('df3', len(df3))
print('df4', len(df4))
print('df5', len(df5))
print('df6', len(df6))
print('df7', len(df7))
print('df8', len(df8))
print('df9', len(df9))
print('df10', len(df10))
print('df11', len(df11))
print('df12', len(df12))
print('df13', len(df13))
print('df14', len(df14))

df1 2656
df2 2564
df3 2546
df4 2606
df5 2825
df6 2253
df7 2648
df8 2613
df9 2656
df10 2560
df11 2563
df12 2522
df13 198
df14 2623


In [66]:
df1 = set(df1)
df2 = set(df2)
df3 = set(df3)
df4 = set(df4)
df5 = set(df5)
df6 = set(df6)
df7 = set(df7)
df8 = set(df8)
df9 = set(df9)
df10 = set(df10)
df11 = set(df11)
df12 = set(df12)
df13 = set(df13)
df14 = set(df14)

In [67]:
a = set.intersection(df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,
                 df11, df12, df13, df14)

In [71]:
final = dart_code_list[dart_code_list['고유번호'].isin(list(a))]
final.reset_index(drop=True).to_json('data/etf_fs_target_company.json')